# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os

# Import API key
from api_keys import g_key


In [ ]:
import gmaps
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

datapath = "../output_data/cities_file"

Cities_df = pd.read_csv(datapath)

Cities_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ambovombe,86,MG,1580273648,77,-25.17,46.08,78.37,7.16
1,1,cap-aux-meules,100,CA,1580273648,83,47.38,-61.86,24.80,17.60
2,2,abu samrah,0,IN,1580273413,31,24.60,72.70,69.80,3.65
3,3,nikolskoye,90,RU,1580273649,86,59.70,30.79,30.20,13.42
4,4,mar del plata,90,ES,1580273649,87,43.40,-4.01,51.01,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:

locations = Cities_df[["Lat", "Lng"]]
humidity = Cities_df['Humidity'].astype(int)


fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

In [ ]:

fig.add_layer(heat_layer)


fig


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [3]:

CityFilter_df = Cities_df.loc[Cities_df['Max Temp'] >= 70,:]
CityFilter_df  = CityFilter_df.loc[CityFilter_df['Max Temp'] <= 80,:]  
CityFilter_df  = CityFilter_df.loc[CityFilter_df['Wind Speed'] <= 10,:]
CityFilter_df = CityFilter_df.loc[CityFilter_df['Cloudiness'] == 0,:]
CityFilter_df = CityFilter_df.dropna()

CityFilter_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
189,189,gursarai,0,IN,1580273671,45,25.61,79.18,71.08,6.58
273,273,khudumelapye,0,BW,1580273683,54,-23.88,24.75,74.10,7.81
311,311,paranaiba,0,BR,1580273688,91,-19.68,-51.19,70.63,5.61
330,330,panaba,0,MX,1580273690,95,21.28,-88.27,70.45,5.23
333,333,hadgaon,0,IN,1580273690,53,19.50,77.67,73.40,5.82
417,417,pokhara,0,NP,1580273700,34,28.23,83.98,70.95,3.44
456,456,bocas del toro,0,CU,1580273706,83,21.07,-76.27,71.60,3.36
506,506,shwebo,0,MM,1580273712,34,22.57,95.70,78.12,4.76


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [4]:
hotel_df = CityFilter_df
hotel_df["Hotel Name"] = " "
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
189,189,gursarai,0,IN,1580273671,45,25.61,79.18,71.08,6.58,
273,273,khudumelapye,0,BW,1580273683,54,-23.88,24.75,74.10,7.81,
311,311,paranaiba,0,BR,1580273688,91,-19.68,-51.19,70.63,5.61,
330,330,panaba,0,MX,1580273690,95,21.28,-88.27,70.45,5.23,
333,333,hadgaon,0,IN,1580273690,53,19.50,77.67,73.40,5.82,
417,417,pokhara,0,NP,1580273700,34,28.23,83.98,70.95,3.44,
456,456,bocas del toro,0,CU,1580273706,83,21.07,-76.27,71.60,3.36,
506,506,shwebo,0,MM,1580273712,34,22.57,95.70,78.12,4.76,


In [5]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
          "radius": 5000,
          "type": "lodging",
          "key": g_key}


hotel_df = hotel_df.reset_index()
Index = list(hotel_df.index.values)
hotel_list = []


for i in Index:
    
    target_coordinates = str(hotel_df['Lat'][i])+' , '+str(hotel_df['Lng'][i])
    params["location"] = target_coordinates
   
    try:
        response = requests.get(base_url, params=params)
        hotels = response.json()
        hotel_list.append(hotels['results'][0]['name'])
        
    except (KeyError, IndexError):
        hotel_list.append(" ")


hotel_df['Hotel Name'] = hotel_list
hotel_df

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,189,189,gursarai,0,IN,1580273671,45,25.61,79.18,71.08,6.58,Kumar ki puliya
1,273,273,khudumelapye,0,BW,1580273683,54,-23.88,24.75,74.10,7.81,
2,311,311,paranaiba,0,BR,1580273688,91,-19.68,-51.19,70.63,5.61,Villa Rica Park Hotel GMR
3,330,330,panaba,0,MX,1580273690,95,21.28,-88.27,70.45,5.23,Hotel California
4,333,333,hadgaon,0,IN,1580273690,53,19.50,77.67,73.40,5.82,Bebabai Kanba Shirasath
5,417,417,pokhara,0,NP,1580273700,34,28.23,83.98,70.95,3.44,Hotel View Point
6,456,456,bocas del toro,0,CU,1580273706,83,21.07,-76.27,71.60,3.36,
7,506,506,shwebo,0,MM,1580273712,34,22.57,95.70,78.12,4.76,MOON SHINE Guest House


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in CityFilter_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

